In [9]:
from openai import OpenAI

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content.rstrip('\n')


def openai_chat(messages):
    client = OpenAI(api_key=read_file("/home/teveritt/OpenAI-API-DCU-AI.key"))

    return client.chat.completions.create(
        model='gpt-4o',
        temperature=0.0,
        messages=messages,
        stream=False,
        max_tokens=1525
    )

In [13]:
def fix_xml(text):
    m = [
        {
            "role": "system",
            "content": """
This GPT will convert XML job postings to plain text. Examples:
    - <company>Company Name: Replace with "Company Name"
    - <company name="Company Name">: Replace with "Company Name"
    - <item>...</item>: Replace with "- ..."
    - <role>...</role>: Replace with "Role: ..."
    - <description>...</description>: Replace with "Description: ..."
    - <job type>Full-Time</job>: Replace with "Job Type: Full-Time"
    - <h>...</h>: Replace with "..."
    - <p>...</p>: Replace with "..."


Apply these replacements to any job posting provided and don't alter anything else.
If there is nothing to convert, then respond with <SKIP>.
"""
        }, {
            "role": "user",
            "content": text
        }
    ]
    output = openai_chat(m)
    content = output.choices[0].message.content.strip()
    content = content.replace('***', '')
    content = content.replace('**', '')
    content = content.replace('*', '-')
    return content

In [14]:
import os

with open('processed-xml.txt', 'r') as file:
    processed = set()
    for line in file.readlines():
        processed.add(line.strip().replace('\n',''))

processed = list(processed)
processed.sort()
print(f'Processed: {processed}')

for type in ['train','val', 'test']:
    dataframes = []
    for file_name in os.listdir(type):
        file_name, file_extension = os.path.splitext(file_name)
        base_path = os.path.join(type, file_name)
        if f'{base_path}.txt' in processed:
            print(f'Already processed {base_path}.txt')
        else:
            if file_name.startswith('Synthetic'):
                if os.path.isfile(f'{base_path}.txt'):
                        print(f'Processing {base_path}.txt')
                        with open(f'{base_path}.txt', 'r') as file:
                            text = file.read().strip()
                        if '<' in text:
                            text = fix_xml(text)
                            if text == '<SKIP>':
                                print(f'Skipping {base_path}.txt')
                            else:
                                print(f'Cleaning {base_path}.txt')
                                with open(f'{base_path}.txt', 'w') as f:
                                    f.write(text)
                            
                            processed.append(f'{base_path}.txt')
                            with open('processed-xml.txt', 'w') as file:
                                for item in processed:
                                    file.write('%s\n' % item)
                            
        #break
    #break
                
